In [1]:
import pandas as pd
import numpy as np
from scipy.stats import wilcoxon
corpus = pd.read_csv('corpus.csv.gz', compression='gzip')
corpus_liwc = pd.read_csv('corpus_liwc_mtx.csv.gz', compression='gzip')

In [2]:
corpus_liwc['class'] = corpus['qual_a_melhor_classificao_para_esse_texto']
corpus_liwc['confidence'] = corpus['qual_a_melhor_classificao_para_esse_texto:confidence']
corpus_liwc['wc'] = corpus['contentcount']
corpus_liwc['judges'] = corpus['_trusted_judgments']

In [3]:
corpus_liwc = corpus_liwc[corpus_liwc.wc.apply(lambda x: str(x).isnumeric())]
corpus_liwc = corpus_liwc[corpus_liwc['judges'] == 3]

In [11]:
diario = corpus_liwc[corpus_liwc['class'] == 'diario'].sample(330)
outro = corpus_liwc[corpus_liwc['class'] == 'outro'].sample(330)

In [12]:
columns = ['funct','pronoun','ppron','i','we','you','shehe','they','ipron','article','verb','auxverb','past','present','future','adverb','preps','conj','negate','quant','number','swear','social','family','friend','humans','affect','posemo','negemo','anx','anger','sad','cogmech','insight','cause','discrep','tentat','certain','inhib','incl','excl','percept','see','hear','feel','bio','body','health','sexual','ingest','relativ','motion','space','time','work','achieve','leisure','home','money','relig','death','assent','nonfl','filler']

In [13]:
stats = pd.DataFrame(data={'diario_mean': diario.mean(axis=0)}, index=columns)
stats['outro_mean'] = outro.mean(axis=0)

In [14]:
significance = []
for column in list(stats.index.values):
    a = diario[column]
    b = outro[column]
    t, p = wilcoxon(a, b)
    significance.append(p)
stats['significance'] = significance

In [25]:
liguistic_columns = ['funct','pronoun','ppron','i','we','you','shehe','they','ipron','article','verb','auxverb','past','present','future','adverb','preps','conj','negate','quant','number']
liguistic_stats = stats.ix[liguistic_columns]
liguistic_stats[['significance']][liguistic_stats.significance > 0.05].sort_values('significance',ascending=False)

,significance
negate,0.948748
past,0.754527
adverb,0.717856
quant,0.217779
number,0.151705
we,0.109697
future,0.102850
verb,0.074561


In [26]:
psychological_columns = ['swear','social','family','friend','humans','affect','posemo','negemo','anx','anger','sad','cogmech','insight','cause','discrep','tentat','certain','inhib','incl','excl','percept','see','hear','feel','bio','body','health','sexual','ingest','relativ','motion','space','time','work','achieve','leisure','home','money','relig','death','assent','nonfl','filler']
psychological_stats = stats.ix[psychological_columns]
psychological_stats[['significance']][psychological_stats.significance > 0.05].sort_values('significance',ascending=False)

,significance
sexual,0.838551
assent,0.820799
feel,0.815046
ingest,0.590055
humans,0.553150
anx,0.548814
home,0.490544
friend,0.466360
sad,0.458225
family,0.425523
